**Set environment**

In [1]:
### set env
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



**Check data**

In [2]:
ls -1 ${FD_RES}/results/region

annotation_chipseq_histone
annotation_chipseq_tf
annotation_chipseq_tf_subset
annotation_chromHMM
annotation_crispri_hcrff
annotation_enhancer_junke
annotation_gencode
annotation_hic
hic_intact_loop
KS91_K562_ASTARRseq_peak_macs_input
region_screened_crispri_flowfish.bed
region_screened_selected.bed
region_screened_selected_perbase.bed
region_screened_selected_perbase.bed.gz
region_screened_tmpra.bed
region_screened_tmpra_crispr.bed


In [3]:
ls -1 ${FD_RES}/results/region/annotation_chipseq_tf | head

accession_info_peaks.tsv
description.tsv
download.sh
K562.ENCSR000AKO.ENCFF769AUF.CTCF.bed.gz
K562.ENCSR000AQA.ENCFF807FNB.KDM5B.bed.gz
K562.ENCSR000AQD.ENCFF408NUX.CHD1.bed.gz
K562.ENCSR000AQE.ENCFF804RVA.EZH2.bed.gz
K562.ENCSR000AQF.ENCFF669MJX.HDAC1.bed.gz
K562.ENCSR000AQG.ENCFF182MPT.HDAC2.bed.gz
K562.ENCSR000AQH.ENCFF981ISM.PHF8.bed.gz


In [5]:
ls -1 ${FD_RES}/results/region/annotation_chipseq_tf/*.bed.gz | wc -l

735


In [6]:
FD_INP=${FD_RES}/results/region/annotation_chipseq_tf
FP_INPS=($(ls ${FD_INP}/K562.*.bed.gz))

echo ${#FP_INPS[@]}
basename -a ${FP_INPS[@]}

735
K562.ENCSR000AKO.ENCFF769AUF.CTCF.bed.gz
K562.ENCSR000AQA.ENCFF807FNB.KDM5B.bed.gz
K562.ENCSR000AQD.ENCFF408NUX.CHD1.bed.gz
K562.ENCSR000AQE.ENCFF804RVA.EZH2.bed.gz
K562.ENCSR000AQF.ENCFF669MJX.HDAC1.bed.gz
K562.ENCSR000AQG.ENCFF182MPT.HDAC2.bed.gz
K562.ENCSR000AQH.ENCFF981ISM.PHF8.bed.gz
K562.ENCSR000AQI.ENCFF942VGF.RBBP5.bed.gz
K562.ENCSR000AQJ.ENCFF383IEP.SAP30.bed.gz
K562.ENCSR000ATJ.ENCFF736FGN.HDAC6.bed.gz
K562.ENCSR000ATL.ENCFF985QBS.CHD4.bed.gz
K562.ENCSR000ATM.ENCFF539MIO.REST.bed.gz
K562.ENCSR000ATT.ENCFF532VPN.CREBBP.bed.gz
K562.ENCSR000ATU.ENCFF258XBJ.CBX2.bed.gz
K562.ENCSR000ATV.ENCFF068OEJ.CBX3.bed.gz
K562.ENCSR000ATW.ENCFF522HZT.CBX8.bed.gz
K562.ENCSR000ATX.ENCFF054XCG.KDM1A.bed.gz
K562.ENCSR000ATY.ENCFF816AEF.NCOR1.bed.gz
K562.ENCSR000ATZ.ENCFF349VSP.KAT2B.bed.gz
K562.ENCSR000AUA.ENCFF923GDR.RNF2.bed.gz
K562.ENCSR000AUB.ENCFF821XJU.SIRT6.bed.gz
K562.ENCSR000AUC.ENCFF647TBC.SUZ12.bed.gz
K562.ENCSR000AUT.ENCFF267QRI.SETDB1.bed.gz
K562.ENCSR000AVD.ENCFF722UJW.CHD7.bed.

In [7]:
FD_INP=${FD_RES}/results/region/annotation_chipseq_tf
FN_INP=K562.ENCSR000BMH.ENCFF398UQZ.YY1.bed.gz
FP_INP=${FD_INP}/${FN_INP}

zcat ${FP_INP} | head

chr1	778486	778776	.	1000	.	132.48683	-1.00000	4.33080	136
chr1	904638	904883	.	1000	.	75.95987	-1.00000	4.33080	112
chr1	913032	913129	.	1000	.	41.88374	-1.00000	3.88398	56
chr1	921136	921238	.	1000	.	50.33578	-1.00000	4.33080	62
chr1	923707	924017	.	1000	.	34.69952	-1.00000	3.76640	155
chr1	924711	925021	.	634	.	20.27464	-1.00000	3.58222	155
chr1	939152	939462	.	1000	.	39.58505	-1.00000	3.91418	155
chr1	939368	939678	.	1000	.	17.19876	-1.00000	3.52219	155
chr1	940224	940534	.	686	.	24.18068	-1.00000	3.70828	155
chr1	941827	942137	.	1000	.	40.54679	-1.00000	3.90079	155


**RUN**

In [18]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 4G \
    --output ${FD_LOG}/annotation_astarr_peak_macs_input_chipseq_tf.%a.txt \
    --array 0-734 \
    <<'EOF'
#!/bin/bash
### set directories
source ../config/config_duke.sh

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set input and output
FD_INP=${FD_RES}/results/region/annotation_chipseq_tf
FP_INPS=($(ls ${FD_INP}/K562.*.bed.gz))
FP_INP=${FP_INPS[SLURM_ARRAY_TASK_ID]}
FN_INP=$(basename ${FP_INP})

FD_REG=${FD_RES}/results/region/KS91_K562_ASTARRseq_peak_macs_input
FN_REG=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
FP_REG=${FD_REG}/${FN_REG}

FD_OUT=${FD_RES}/results/region/KS91_K562_ASTARRseq_peak_macs_input/annotation_chipseq_tf
FN_OUT=peak.annotation.${FN_INP}
FP_OUT=${FD_OUT}/${FN_OUT}

### show I/O file
echo "Input: " ${FP_REG}
echo
echo "show first few lines of input"
cat ${FP_REG} | head -5
echo
echo "Input: " ${FP_INP}
echo
echo "show first few lines of input"
zcat ${FP_INP} | head -5
echo

### init: create output folder if not exist
mkdir -p ${FD_OUT}

### annotation using intersect
bedtools intersect \
    -a ${FP_REG} \
    -b ${FP_INP} \
    -wo \
    -f 0.7 \
    -F 0.7 \
    -e \
| gzip -c \
> ${FP_OUT}

### show output file
echo
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of output:"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29724502


**Check Results**

In [19]:
cat ${FD_LOG}/annotation_astarr_peak_macs_input_chipseq_tf.0.txt

Hostname:           x3-01-4.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         04-24-23+15:38:11

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_chipseq_tf/K562.ENCSR000AKO.ENCFF769AUF.CTCF.bed.gz

show first few lines of input
chr1	267966	268092	.	1000	.	36.26682	-1.00000	4.33816	58
chr1	778701	778979	.	1000	.	78.32446	-1.00000	4.65032	175
chr1	778870	779294	.	1000	.	22.45062	-1.00000	4.08853	212
chr1	869772	870056	.	1000	.	140.67100	-1.00000	4.65032	149
chr1	904625	904954	.	1000	.	150.99362	-1.00000	4.65032	154


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/annotation_chipseq_tf/pea

In [20]:
cat ${FD_LOG}/annotation_astarr_peak_macs_input_chipseq_tf.300.txt

Hostname:           x3-04-2.genome.duke.edu
Slurm Array Index:  300
Time Stamp:         04-24-23+15:39:41

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_chipseq_tf/K562.ENCSR201DCF.ENCFF718IGO.ZFP91.bed.gz

show first few lines of input
chr1	906732	907092	.	1000	.	165.26358	-1.00000	3.31660	180
chr1	921156	921516	.	820	.	38.15332	-1.00000	2.09263	180
chr1	921361	921721	.	629	.	12.70554	-1.00000	0.03000	180
chr1	969416	969776	.	613	.	26.50667	-1.00000	0.98611	180
chr1	1000678	1001038	.	1000	.	48.69294	-1.00000	2.51870	180


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/annotation_chipseq_tf/p

**Problem: peak.annotation.K562.ENCSR633EIC.ENCFF841RMH.PBX2.bed.gz**
```
Error: line number 3861 of file /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_chipseq_tf/K562.ENCSR633EIC.ENCFF841RMH.PBX2.bed.gz has 8 fields, but 10 were expected.
```

In [29]:
cat ${FD_LOG}/annotation_astarr_peak_macs_input_chipseq_tf.528.txt

Hostname:           x3-03-3.genome.duke.edu
Slurm Array Index:  528
Time Stamp:         04-24-23+15:41:12

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_chipseq_tf/K562.ENCSR633EIC.ENCFF841RMH.PBX2.bed.gz

show first few lines of input
chr1	921034	921274	.	1000	.	42.08720	-1.00000	2.67091	120
chr1	940127	940367	.	639	.	32.89746	-1.00000	2.38666	120
chr1	960486	960726	.	677	.	20.97131	-1.00000	0.93739	120
chr1	1000106	1000346	.	1000	.	59.60702	-1.00000	3.42472	120
chr1	1033088	1033328	.	1000	.	44.46317	-1.00000	2.64864	120

Error: line number 3861 of file /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_chipseq_tf/K562.ENCSR633E

In [30]:
zcat /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_chipseq_tf/K562.ENCSR633EIC.ENCFF841RMH.PBX2.bed.gz | head -n 3865 | tail -n 5

chrX	47650630	47650870	.	650	.	12.29068	-1.00000	-0.07259	120
chrX	48071804	48072044	.	1000	.	61.98283	-1.00000	3.42472	120
chrX	48696365	48696605	.	1000	.	52.42240	-1.00000	2.95085	120
chrX	48800567	48800807	.	1000	.	41.76349	-1.00000	2.67486	120
chrX	48958365	48958605	.	1000	.	89.01953	-1.00000	3.42472	120


In [31]:
zcat /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/annotation_chipseq_tf/peak.annotation.K562.ENCSR633EIC.ENCFF841RMH.PBX2.bed.gz

In [21]:
cat ${FD_LOG}/annotation_astarr_peak_macs_input_chipseq_tf.600.txt

Hostname:           x3-03-1.genome.duke.edu
Slurm Array Index:  600
Time Stamp:         04-24-23+15:41:42

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_chipseq_tf/K562.ENCSR764OXF.ENCFF983LFS.RBM39.bed.gz

show first few lines of input
chr1	1375402	1375822	.	723	.	26.44600	-1.00000	2.07377	210
chr1	1401405	1401825	.	622	.	21.53238	-1.00000	1.69463	210
chr1	1401632	1402052	.	622	.	16.53055	-1.00000	1.12617	210
chr1	1407920	1408340	.	550	.	26.47169	-1.00000	2.07155	210
chr1	1614299	1614719	.	551	.	21.60066	-1.00000	1.69414	210


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/annotation_chipseq

In [22]:
cat ${FD_LOG}/annotation_astarr_peak_macs_input_chipseq_tf.734.txt

Hostname:           x3-05-2.genome.duke.edu
Slurm Array Index:  734
Time Stamp:         04-24-23+15:42:12

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_chipseq_tf/K562.ENCSR998OMC.ENCFF392MAR.TFCP2.bed.gz

show first few lines of input
chr1	778654	778984	.	1000	.	61.96326	-1.00000	2.23598	165
chr1	958976	959306	.	777	.	47.57064	-1.00000	1.54895	165
chr1	976024	976354	.	555	.	41.51387	-1.00000	1.11517	165
chr1	999276	999606	.	870	.	53.99746	-1.00000	1.91586	165
chr1	1000723	1001053	.	761	.	64.59282	-1.00000	2.30012	165


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/annotation_chipseq_tf/pea

In [26]:
FDIRY=/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/annotation_chipseq_tf
FPATH=$(ls ${FDIRY}/peak.*.FOS.bed.gz)
echo ${FPATH}

/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/annotation_chipseq_tf/peak.annotation.K562.ENCSR000DKB.NA.FOS.bed.gz /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/annotation_chipseq_tf/peak.annotation.K562.ENCSR000FAI.ENCFF258PLH.FOS.bed.gz


In [27]:
FNAME=peak.annotation.K562.ENCSR000DKB.NA.FOS.bed.gz
FPATH=${FDIRY}/${FNAME}
zcat ${FPATH} 

In [28]:
FNAME=peak.annotation.K562.ENCSR000FAI.ENCFF258PLH.FOS.bed.gz
FPATH=${FDIRY}/${FNAME}
zcat ${FPATH}  | head

chr1	777949	779437	chr1	778609	778873	.	1000	.	50.97238	-1.00000	4.46593	132	264
chr1	1063757	1064554	chr1	1064241	1064505	.	1000	.	41.64451	-1.00000	4.46593	132	264
chr1	1207889	1208506	chr1	1208029	1208293	.	989	.	22.70364	-1.00000	4.46593	132	264
chr1	1230796	1232701	chr1	1232028	1232202	.	1000	.	137.03011	-1.00000	4.46593	96	174
chr1	1317989	1318547	chr1	1318284	1318548	.	1000	.	39.45642	-1.00000	4.46593	132	263
chr1	1324260	1325767	chr1	1324577	1324841	.	1000	.	44.42891	-1.00000	4.46593	132	264
chr1	1344582	1346157	chr1	1344867	1345131	.	785	.	22.25587	-1.00000	4.46593	132	264
chr1	1398920	1400474	chr1	1399296	1399555	.	1000	.	515.08126	-1.00000	4.46593	126	259
chr1	1470698	1473196	chr1	1471567	1471831	.	1000	.	32.86756	-1.00000	4.46593	132	264
chr1	1511039	1512740	chr1	1511961	1512225	.	816	.	31.53213	-1.00000	4.46593	132	264
